# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts url in it and check which other subreddit reposted that posts and how much:

In [ ]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

for i in reddit.subreddit("conspiracy").top(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        subreddit_url = str(repost.subreddit)
        subreddit_url = "https://www.reddit.com/r/" + subreddit_url
        if subreddit_url in conspiracy_dict.keys():
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            conspiracy_dict[subreddit_url][1][0] +=1
        else:
            conspiracy_dict[subreddit_url]=[[],[1]]
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_data/conspiracy_top_url.csv',index=False)




# Creating the network
Now we need to create the network of subreddits involved.

In [ ]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

network_list = list()

for subr in datasets:
        #print(f'now opening {subr}')
        try:
                df1 = pd.read_csv(subr)
        except:
                pass
        try:
                df1 = pd.read_csv(subr, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        sub_name=subr[subr.find("\\")+1:-4]
        for column in df1.columns:
                if column [:7] != "Unnamed":
                        col_name = column[column.find("/r/")+3:]
                        try:
                                network_list.append((sub_name, col_name, df1[column][1]))
                        except Exception as E:
                                print(E) 
                else: 
                        continue
print(network_list)

'''with open('Network.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(["Source", "Target", "Weight"])
    write.writerows(network_list)'''

# Looking for subnetworks in csv

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [12]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

networks=[file for file in get_all_in_dir("results/subnetworks")]

for net in networks:
        try:
                df_network = pd.read_csv(net)
        except:
                pass
        try:
                df_network = pd.read_csv(net, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        network_name=net[net.find("\\")+1:-4]
        for subr in datasets:
                #print(f'now opening {subr}')
                try:
                        df_dataset = pd.read_csv(subr)
                except:
                        pass
                try:
                        df_dataset = pd.read_csv(subr, encoding='utf8')
                except:
                        print(f'unable to open {subr}')
                        continue
                sub_name=subr[subr.find("\\")+1:-4]
                for column in df_dataset.columns:
                        if column [:7] != "Unnamed":
                                col_name = column[column.find("/r/")+3:]
                                for index, row in df_network.iterrows():
                                        if (col_name == row["Source"] and sub_name == row["Target"]) or (col_name == row["Target"] and sub_name == row["Source"]):
                                                print(col_name,sub_name, row["Source"], row["Target"])
                                                try:
                                                        network_list.append((sub_name, col_name, df_dataset[column][1],df_dataset[column][0]))
                                                except Exception as E:
                                                        print(E) 
                                        else: 
                                                continue
        
        
        with open(network_name+'.csv', 'w') as f:
                # using csv.writer method from CSV package
                write = csv.writer(f)
                
                write.writerow(["Source", "Target", "Weight","Comments"])
                write.writerows(network_list)
                
                        

KeyboardInterrupt: 

In [ ]:
#n most connected subreddits
def takeSecond(elem):
    return int(elem[2])

sorted(network_list, key=takeSecond, reverse=True)